<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0120onomatopea_bert_fine_turing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- filename: 2022_01202onomatopea_bert_fine_tuing.ipynb
- memo: 2022年01月20日現在，

transformers は M1 Mac では動作しない。Intel Mac such as pasiphae では動作する。

In [1]:
import os
import sys
import numpy as np
import unicodedata
from termcolor import colored

# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストールする
import platform
isColab = platform.system() == 'Linux'
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1
    !pip install jaconv > /dev/null 2>&1 

In [3]:
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み

#'日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードしてください
if isColab:
    from google.colab import files
    uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
    data_dir = '.'
else:
    data_dir = '/Users/asakawa/study/2021ccap/notebooks'

import pandas as pd
import jaconv

onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(data_dir, onomatopea_excel), sheet_name='2761語')


#すべてカタカナ表記にしてデータとして利用する場合
#`日本語オノマトペ辞典4500` はすべてひらがな表記だが，一般にオノマトペはカタカナ表記されることが多いはず
#onomatopea = list(sorted(set([jaconv.hira2kata(o) for o in onmtp2761['オノマトペ']])))
# Mac と Windows の表記の相違を吸収
onomatopea = list(sorted(set([jaconv.normalize(o) for o in onmtp2761['オノマトペ']])))
print(f'データファイル名: {os.path.join(data_dir, onomatopea_excel)}\n',
      f'オノマトペ単語総数: len(onomatopea):{len(onomatopea)}')

Saving 2021-0325日本語オノマトペ辞典4500より.xls to 2021-0325日本語オノマトペ辞典4500より.xls
データファイル名: ./2021-0325日本語オノマトペ辞典4500より.xls
 オノマトペ単語総数: len(onomatopea):1741


In [4]:
# 近藤先生 (2021年12月22日） から送っていただいた，オノマトペ文章データ 'original.csv' を読み込む
import jaconv

if isColab:
    uploaded = files.upload()  # original.csv をアップロード
    data_dir = '.'
else:
    data_dir = '/Users/asakawa/study/2021kondo_project'

original = []
n = 0
with open(os.path.join(data_dir,'original.csv'), 'r', encoding='utf8') as f:
    s = f.read()
    for s_ in s.split('\n'):
        if n == 0:
            n += 1
            continue
        idx, sent = s_.split(',')
        
        # Mac と Windows との unicode 符号化の差分を吸収する
        # jaconv.normalize は内部で unicodedata.normalize('NFKC') を呼び出しているので
        # 差異 between Mac and Windows を吸収できる
        sent = ''.join(jaconv.normalize(x) for x in sent)
        original.append(sent)
        #original[int(idx)] = sent

print(f'{len(original)} has been read')

Saving original.csv to original.csv
2469 has been read


In [5]:
%%time
import torch
from transformers import BertConfig
#from transformers import BertModel
from transformers import BertForPreTraining
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM


model_ja_name = 'cl-tohoku/bert-base-japanese' 
model = BertForMaskedLM.from_pretrained(model_ja_name)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

config = BertConfig.from_pretrained(model_ja_name)

# トークナイザ の修正
tknz1 = BertJapaneseTokenizer.from_pretrained(model_ja_name)
# BPE (or sentencepiece) による下位単語分割あり

tknz1.add_tokens(onomatopea)
model.resize_token_embeddings(len(tknz1))

print(f' len(tknz1):{len(tknz1)}\n', 
      f'len(tknz1.vocab):{len(tknz1.vocab)}\n',  # 一見すると，この数字からオノマトペが追加されていないように見える。
      f'tknz1.vocab_size:{tknz1.vocab_size}')    # 駄菓子菓子，下で見るように，正しく動作しているように見受けられる

# 確認用
for w in onomatopea[-5:]:
    idx = tknz1.convert_tokens_to_ids(w)
    w_ = tknz1.convert_ids_to_tokens(idx)
    print(f'{w} id:{idx} -> token:{w_}')
    

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/104 [00:00<?, ?B/s]

 len(tknz1):33711
 len(tknz1.vocab):32000
 tknz1.vocab_size:32000
わんわ id:33706 -> token:わんわ
わんわん id:33707 -> token:わんわん
わーっ id:33708 -> token:わーっ
わーわー id:33709 -> token:わーわー
わーん id:33710 -> token:わーん
CPU times: user 11.3 s, sys: 3.14 s, total: 14.5 s
Wall time: 30.4 s


In [6]:
# こちらも確認用，オノマトペを追加していないトークナイザwを tknz2 とする
tknz2 = BertJapaneseTokenizer.from_pretrained(model_ja_name)
print(tknz1.tokenize('雨がしとしとと降る'))  #オノマトペ追加バージョンの出力
print(tknz2.tokenize('雨がしとしとと降る'))  #without オノマトペのトークナイザによる出力

['雨', 'が', 'しとしと', 'と', '降る']
['雨', 'が', 'し', '##と', '##し', '##と', '##と', '降る']


In [7]:
import re
# ランダムサンプリングしてデータを印字して確認
for _ in range(3):
    N = np.random.randint(low=0, high=len(original))
    sent0 = original[N]
    sent1 = re.sub('\(と\)','と',original[N]) # original に含まれる `(と)` のような表現を削除する
    
    print(colored(sent0, attrs=['bold']))  # 送っていただいた元の文
    print(colored('\t分かち書き','blue'), tknz1.tokenize(sent0)) # その分かち書き
    print(colored('\tトークン ID', 'blue'), tknz1.encode(sent0))     # 分かち書き結果の単語 ID 化

    if sent0 != sent1:
        print(colored('\t分かち書き','red'), tknz1.tokenize(sent1)) # その分かち書き
        print(colored('\tトークンID', 'red'), tknz1.encode(sent1))      # 分かち書き結果の単語 ID 化

# MeCab で単語分割が行われて、MeCab が単語として認識しても、その単語が語鎮リスト vocab.txt に登録されていない場合は
# subword である WordPiece が起動され、その単語が適当に分割されます。そのように分割された単語には '##' が単語の前に付与されます。
# また、未知語の場合もWordPieceが起動され、同様に分割されます。

print('\n', '-' * 77)
print('# 以下は，特殊トークンと対応するトークン ID との関係を表示。')
print('# 英語版で標準的に用いられる `bert-base-uncased` と東北大学乾研の特殊トークンの ID は異なることに注意')
print(colored(f'tknz.all_special_ids:{tknz1.all_special_ids}',attrs=['bold']))  #  [1, 3, 0, 2, 4]
print(colored(f'tknz.all_special_tokens:{tknz1.all_special_tokens}', attrs=['bold']))  #  ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

重いものがごとんと倒れたり落ちる
	分かち書き ['重い', 'もの', 'が', 'ごとん', 'と', '倒れ', 'たり', '落ちる']
	トークン ID [2, 10403, 120, 14, 32514, 13, 8390, 790, 13775, 3]
かたく薄いものがばきん と急に折れる
	分かち書き ['かた', '##く', '薄い', 'もの', 'が', 'ばきん', 'と', '急', 'に', '折れ', '##る']
	トークン ID [2, 6223, 28504, 10505, 120, 14, 33092, 13, 1132, 7, 16640, 28449, 3]
かたいものでこんこん(と)何度もたたく
	分かち書き ['かた', '##い', 'もの', 'で', 'こんこん', '(', 'と', ')', '何', '度', 'も', 'たた', '##く']
	トークン ID [2, 6223, 28457, 120, 12, 32469, 23, 13, 24, 1037, 559, 28, 11298, 28504, 3]
	分かち書き ['かた', '##い', 'もの', 'で', 'こんこん', 'と', '何', '度', 'も', 'たた', '##く']
	トークンID [2, 6223, 28457, 120, 12, 32469, 13, 1037, 559, 28, 11298, 28504, 3]

 -----------------------------------------------------------------------------
# 以下は，特殊トークンと対応するトークン ID との関係を表示。
# 英語版で標準的に用いられる `bert-base-uncased` と東北大学乾研の特殊トークンの ID は異なることに注意
tknz.all_special_ids:[1, 3, 0, 2, 4]
tknz.all_special_tokens:['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [8]:
text = tuple(original)  # 全文をタプルに変換

# トークナイザにかけて出力を得る。`max_length` のデフォルトは 512 だが，今回は長文である必要がないと考えられる
# ので 32 にしている。512 でも動作するが，学習に要する時間が増える
inputs = tknz1(text, return_tensors='pt', max_length=32, truncation=True, padding='max_length')

#`labels` キーを追加する。実際には inputs_ids なのでラベルではなくトークンID の系列
inputs['labels'] = inputs.input_ids.detach().clone()

#トークン ID を走査して，オノマトペ単語であれば，[MASK] トークンに置き換える。
l_ = []
for l in inputs['labels']:
    l_.append([tknz1.mask_token_id if w in onomatopea else tknz1.convert_tokens_to_ids(w) for w in tknz1.convert_ids_to_tokens(l)])

inputs['input_ids'] = torch.LongTensor(l_)
#print(inputs['input_ids'].shape)

In [ ]:
#%%time
#print(f'# 直上で定義した `inputs` は全オノマトペの著者，小野による説明文すべてである。総数は {len(inputs.input_ids)}')
#print('これを一通り評価するにはどれほどの時間がかかkるのか，時間を計測してみる')
#outputs = model(**inputs)

In [ ]:
#print(inputs['input_ids'][0][:32])
#print(inputs['labels'][0][:10])

In [ ]:
#データセットのためのクラスを定義
class onmtpDataset(torch.utils.data.Dataset):
    def __init__(self, encoder):
        self.encoder = encoder
        
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encoder.items()}
    
    def __len__(self):
        return len(self.encoder.input_ids)
    
dataset = onmtpDataset(inputs)

#データローダを準備
loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

# GPU/CPU 使用を設定し，モデルの訓練モードを起動 #Setup GPU/CPU usage and activate the training mode of our model.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# そしてモデルを選択したデバイスに移動 # and move our model over to the selected device
model.to(device)
# 訓練モードに設定 #activate training mode
model.train()

In [10]:
#最適化関数を初期化 (AdamW は重み付き崩壊で，過学習の可能性を減らします) 
#Initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

from transformers import AdamW
#最適化関数を初期化 # initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [11]:
from tqdm import tqdm  # for our progress bar

epochs = 5

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/618 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
Epoch 4: 100%|██████████| 618/618 [00:31<00:00, 19.68it/s, loss=0.118]


In [59]:
#tknz1.convert_ids_to_tokens(
a = input_ids.detach().squeeze()
print(tknz1.convert_ids_to_tokens(a))
x = outputs.logits.detach()

print('-' * 77)
_x = x.squeeze(0).detach().clone()
print(tknz1.convert_ids_to_tokens(torch.argmax(_x, dim=1, keepdim=True)))
#help(torch.argmax)

['[CLS]', '大', '##粒', 'の', '雨', 'や', '雷', 'など', 'が', '[MASK]', '(', 'と', ')', '急', 'に', '降る', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
-----------------------------------------------------------------------------
['[CLS]', '大', '##粒', 'の', '雨', 'や', '雷', 'など', 'が', 'ごろごろ', '(', 'と', ')', '急', 'に', '降る', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [60]:
inputs['input_ids'].shape

torch.Size([2469, 32])

In [ ]:
outputs.logits.shape

In [ ]:
print(tknz1.tokenize('日が射したり光が[MASK]とともる'))